In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph

class State(TypedDict):
    counter: int
    alphaber: list[str]

graph_builder = StateGraph(State)

In [9]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
import operator
class State(TypedDict):
    counter: int
    alphabet: list[str]

def node_a(state:State):
    state['counter'] += 1
    state['alphabet'] = ['Hello']
    return state

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", node_a)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [10]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermade_png()))
except Exception:
    print(Exception)
    pass

<class 'Exception'>


In [11]:
initial_state={
    "counter": 0,
    "alphabet": []
}

state=initial_state
for _ in range(3):
    state=graph.invoke(state)
    print(state)

{'counter': 1, 'alphabet': ['Hello']}
{'counter': 2, 'alphabet': ['Hello']}
{'counter': 3, 'alphabet': ['Hello']}


In [13]:
class State(TypedDict):
    counter: int
    alphabet: Annotated[list[str], operator.add]

def node_a(state: State):
    state['counter'] += 1
    state['alphabet'] = ['Hello']
    return state

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", node_a)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [14]:
initial_state={
    "counter": 0,
    "alphabet": []
}

state=initial_state
for _ in range(3):
    state=graph.invoke(state)
    print(state)

{'counter': 1, 'alphabet': ['Hello']}
{'counter': 2, 'alphabet': ['Hello', 'Hello']}
{'counter': 3, 'alphabet': ['Hello', 'Hello', 'Hello']}


In [19]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
llm=ChatOpenAI(model = "gpt-4o-mini")

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph=graph_builder.compile()

In [20]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 안녕하세요! 어떻게 도와드릴까요?
Assistant: 죄송하지만, 현재 날씨 정보를 직접 제공할 수는 없습니다. 하지만, 특정 지역의 날씨를 확인하고 싶으시면 기상청 웹사이트나 날씨 앱을 이용해 보시는 것을 추천합니다. 도움이 필요하시면 말씀해 주세요!
Assistant: 저는 감정을 느끼지 않지만, 당신의 기분이나 상황에 대해 이야기해주시면 도움이 될 수 있습니다! 어떤 이야기를 나누고 싶으신가요?
Goodbye!


In [22]:
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState

graph_builder = StateGraph(MessagesState)
llm = ChatOpenAI(model = "gpt-4o-mini")

def chatbot(state:State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot",chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph=graph_builder.compile()

In [23]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 안녕하세요! 어떻게 도와드릴까요?
Goodbye!


In [24]:
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState

class State(MessagesState):
    counter: int

graph_builder = StateGraph(State)
llm = ChatOpenAI(model = "gpt-4o-mini")

def chatbot(state:State):
    state["counter"] = state.get('counter', 0)+1
    return {
        "counter": state["counter"],
        "messages": [llm.invoke(state["messages"])]
    }

graph_builder.add_node("chatbot",chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph=graph_builder.compile()

In [26]:
from langchain_core.messages import HumanMessage

initial_state={
    "messages": [HumanMessage(content="Hello!")],
    "counter": 0
}

result = graph.invoke(initial_state)
print(f"Final state: {result}")

Final state: {'messages': [HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='62bc9d7d-3120-4adb-9d22-f94e669e98e4'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-9cbb0c3d-6097-4ff1-9767-016296cdcc1a-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})], 'counter': 1}


In [27]:
state = initial_state
for _ in range(3):
    state = graph.invoke(state)
    print(f"Counter: {state['counter']}")
    print(f"Last Message: {state['messages'][-1].content}")
    print("---")

Counter: 1
Last Message: Hello! How can I assist you today?
---
Counter: 2
Last Message: Hello! How can I assist you today?
---
Counter: 3
Last Message: Hello! How can I assist you today?
---


: 